## Initiate the vissim instance

In [1]:
# COM-Server
import win32com.client as com
import igraph
import qgrid
from VISSIM_helpers import VissimRoadNet
from os.path import abspath, join, exists
import os
from shutil import copyfile
import pandas as pd
import math
from pythoncom import com_error

Add autocompletion for VISSIM COM Object

In [2]:
from IPython.utils.generics import complete_object

@complete_object.register(com.DispatchBaseClass)
def complete_dispatch_base_class(obj, prev_completions):
    try:
        ole_props = set(obj._prop_map_get_).union(set(obj._prop_map_put_))
        return list(ole_props) + prev_completions
    except AttributeError:
        pass

Start Vissim and load constants

In [3]:
Vissim = com.gencache.EnsureDispatch("Vissim.Vissim")
from win32com.client import constants as c

Setting the parameters used for simulation

In [4]:
DTA_Parameters = dict(
    # DTA Parameters
    EvalInt = 600, # seconds
    ScaleTotVol = False,
    ScaleTotVolPerc = 1,
    CostFile = 'costs.bew',
    ChkEdgOnReadingCostFile = True,
    PathFile = 'paths.weg',
    ChkEdgOnReadingPathFile = True,
    CreateArchiveFiles = True,
    VehClasses = '',
)

# Simulation parameters
Sim_Parameters = dict(
    NumRuns = 1,
    RandSeedIncr = 0,
    UseMaxSimSpeed = True,
    SimBreakAt = 600,
    NumCores = 8,
)

FileName = abspath(r"..\SO sim files\Vol100per.inpx")
WorkingFolder = abspath(r"..\SO sim files")

In [5]:
def current_period():
    return int(math.ceil(Vissim.Simulation.SimulationSecond / DTA_Parameters['EvalInt']))

Resetting edge and path cost files

In [6]:
default_cost_file = abspath('..\SO sim files\costs_020.bew')
defualt_path_file = abspath('..\SO sim files\paths_020.weg')

current_cost_file = abspath(join(WorkingFolder, DTA_Parameters['CostFile']))
if exists(current_cost_file):
    os.remove(current_cost_file)
copyfile(default_cost_file, current_cost_file)

current_path_file = abspath(join(WorkingFolder, DTA_Parameters['PathFile']))
if exists(current_path_file):
    os.remove(current_path_file)
copyfile(defualt_path_file, current_path_file)

'C:\\Users\\ITSLab\\Documents\\Oliver Liang\\SO sim files\\paths.weg'

Load the test network

In [7]:
Vissim.LoadNet(FileName)

Read dynamic assignment network

In [8]:
vis_net = Vissim.Net
vis_net.Paths.ReadDynAssignPathFile()

In [9]:
network_graph = VissimRoadNet(vis_net)

Check if dynamic assignment graph has changed

In [10]:
ref_edge_list = pd.read_pickle("edges_attr.pkl.gz")
assert (network_graph.visedges['ToNode'] == ref_edge_list['ToNode']).all()
network_graph.save(join(WorkingFolder, "network_graph.pkl.gz"), format="picklez")

We start by opening the network to be tested and adjust its settings

In [11]:
DynamicAssignment = Vissim.Net.DynamicAssignment
for attname, attvalue in DTA_Parameters.items():
    DynamicAssignment.SetAttValue(attname, attvalue)
    
Simulation = Vissim.Net.Simulation
for attname, attvalue in Sim_Parameters.items():
    Simulation.SetAttValue(attname, attvalue)

Run first DTA period as usual

In [12]:
Vissim.Graphics.CurrentNetworkWindow.SetAttValue("QuickMode", 1)
Simulation.RunSingleStep()
while current_period() < 2:
    network_graph.update_volume(vis_net)
    Simulation.RunSingleStep()

Run simulation with custom route assignment

In [ ]:
bad_paths = []
while True:
    network_graph.update_weights(vis_net)
    new_vehs = vis_net.Vehicles.GetDeparted()
    for veh in new_vehs:
        origin_lot = int(veh.AttValue('OrigParkLot'))
        destination_lot = int(veh.AttValue('DestParkLot'))
        node_paths, edge_paths  = network_graph.parking_lot_routes(origin_lot, destination_lot)
        try:
            vis_path = vis_net.Paths.AddPath(origin_lot, destination_lot, [str(node) for node in node_paths[0]])
            veh.AssignPath(vis_path)
        except com_error:
            bad_paths.append((node_paths[0], edge_paths[0]))
    network_graph.update_volume(vis_net)
    if Vissim.Simulation.SimulationSecond > 4499:
        break
    Vissim.Simulation.RunSingleStep()

Vissim.Simulation.RunContinuous()

In [32]:
vis_net.Paths.AddPath(origin_lot, destination_lot, [str(node) for node in node_paths[0]])

com_error: (-2147352567, 'Exception occurred.', (0, 'VISSIM.Vissim.900', 'AddPath failed: adjacent node pair in node list with no open edge between nodes.', None, 0, -2147352567), None)

In [33]:
veh.AttValue('No')

32528

In [37]:
from pythoncom import com_error

In [35]:
node_paths[0]

[1000077,
 1000078,
 1757,
 1000098,
 1000070,
 45,
 1000069,
 1000068,
 1000099,
 1000102,
 1000101,
 1000100,
 1000099,
 1000068,
 1000069,
 45,
 1000133,
 1000067,
 47,
 1000057,
 1000121,
 1000132]

In [28]:
edge_weights = network_graph.es[[ed - 1 for ed in edge_paths[0]]]['weight']
print(sum(edge_weights))
pd.DataFrame(list(zip(edge_paths[0], edge_weights)), columns=['edge', 'edge_weights'])

752.8113709490541


,edge,edge_weights
0,31,15.252632
1,1070,21.212500
2,340,6.721250
3,1077,24.575439
4,341,53.463107
5,598,47.222857
6,52,8.045455
7,1159,2.643243
8,386,25.800000
9,1029,21.800000


In [29]:
edges = [int(ed) for ed in veh.Path.AttValue('EdgeSeq').split(',')]
edge_weights = network_graph.es[[ed - 1 for ed in edges]]['weight']
print(sum(edge_weights))
pd.DataFrame(list(zip(edges, edge_weights)), columns=['edge', 'edge_weights'])

1302.5067067757254


,edge,edge_weights
0,31,15.252632
1,1070,21.212500
2,340,6.721250
3,1077,24.575439
4,341,53.463107
5,596,96.156604
6,48,35.200000
7,1044,42.200000
8,328,59.600000
9,1042,72.000000


In [ ]:
Vissim.Simulation.RunContinuous()

In [ ]:
Vissim.Exit()